# Prepare Datasets for Join

Let us prepare datasets to Join
- Make sure airport-codes is in HDFS
- We will also use airlines data for the month of January 2008. We have used that data set in the past as well.

In [1]:
import sys.process._

In [2]:
"hdfs dfs -ls /public/airlines_all/" !

Found 3 items
drwxr-xr-x   - hdfs hdfs          0 2019-11-21 10:07 /public/airlines_all/airlines
drwxr-xr-x   - hdfs hdfs          0 2020-03-11 13:15 /public/airlines_all/airlines-part
drwxr-xr-x   - hdfs hdfs          0 2019-11-26 18:32 /public/airlines_all/airport-codes


0

In [ ]:
"hdfs dfs -ls /public/airlines_all/airlines-part" !

## Starting Spark Context

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.
    builder.
    config("spark.ui.port","0").
    appName("Basic Transformations").
    master("yarn").
    getOrCreate

spark = org.apache.spark.sql.SparkSession@1373c13f


org.apache.spark.sql.SparkSession@1373c13f

In [3]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [4]:
import spark.implicits._

## Analyze Datasets for Join

Let us analyze data sets that are gooing to be used for joins 

- We will use January 2008 airlines dat which have all relevant details
- Let us read and review the airlines data quickly

In [1]:
val airlines = spark.
    read.
    parquet("/public/airlines_all/airlines-part/flightmonth=200801")

airlines = [Year: int, Month: int ... 29 more fields]


[Year: int, Month: int ... 29 more fields]

In [8]:
airlines.printSchema

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

- We will be using another data set to get details about airports. Details include infirmation such as State, City for given airport code
- Let us analyze the Dataset to confirm if there is header and also how the data is structured.

In [16]:
"hdfs dfs -ls /public/airlines_all/airport-codes" !

Found 1 items
-rw-r--r--   2 hdfs hdfs      11411 2019-11-26 18:32 /public/airlines_all/airport-codes/airport-codes-na.txt


0

In [5]:
val airportCodesPath = "/public/airlines_all/airport-codes"

airportCodesPath = /public/airlines_all/airport-codes


/public/airlines_all/airport-codes

In [7]:
spark.
    read.
    text(airportCodesPath).
    show(false)

+-------------------------+
|value                    |
+-------------------------+
|City	State	Country	IATA  |
|Abbotsford	BC	Canada	YXX |
|Aberdeen	SD	USA	ABR      |
|Abilene	TX	USA	ABI       |
|Akron	OH	USA	CAK         |
|Alamosa	CO	USA	ALS       |
|Albany	GA	USA	ABY        |
|Albany	NY	USA	ALB        |
|Albuquerque	NM	USA	ABQ   |
|Alexandria	LA	USA	AEX    |
|Allentown	PA	USA	ABE     |
|Alliance	NE	USA	AIA      |
|Alpena	MI	USA	APN        |
|Altoona	PA	USA	AOO       |
|Amarillo	TX	USA	AMA      |
|Anahim Lake	BC	Canada	YAA|
|Anchorage	AK	USA	ANC     |
|Appleton	WI	USA	ATW      |
|Arviat	NWT	Canada	YEK    |
|Asheville	NC	USA	AVL     |
+-------------------------+
only showing top 20 rows



- Data is tab seperated
- There is header for the data set.
- Dataset have 4 fields - **Country, State, City, IATA**

In [8]:
val airportCodesPath = "/public/airlines_all/airport-codes"

airportCodesPath = /public/airlines_all/airport-codes


/public/airlines_all/airport-codes

In [9]:
val airportCodes = spark.
    read.
    option("sep", "\t").
    option("header", true).
    option("inferSchema", true).
    csv(airportCodesPath)

airportCodes = [City: string, State: string ... 2 more fields]


[City: string, State: string ... 2 more fields]

In [10]:
airportCodes.printSchema

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- IATA: string (nullable = true)



- Preview and understand the data

In [25]:
airportCodes.show(false)

+-----------+-----+-------+----+
|City       |State|Country|IATA|
+-----------+-----+-------+----+
|Abbotsford |BC   |Canada |YXX |
|Aberdeen   |SD   |USA    |ABR |
|Abilene    |TX   |USA    |ABI |
|Akron      |OH   |USA    |CAK |
|Alamosa    |CO   |USA    |ALS |
|Albany     |GA   |USA    |ABY |
|Albany     |NY   |USA    |ALB |
|Albuquerque|NM   |USA    |ABQ |
|Alexandria |LA   |USA    |AEX |
|Allentown  |PA   |USA    |ABE |
|Alliance   |NE   |USA    |AIA |
|Alpena     |MI   |USA    |APN |
|Altoona    |PA   |USA    |AOO |
|Amarillo   |TX   |USA    |AMA |
|Anahim Lake|BC   |Canada |YAA |
|Anchorage  |AK   |USA    |ANC |
|Appleton   |WI   |USA    |ATW |
|Arviat     |NWT  |Canada |YEK |
|Asheville  |NC   |USA    |AVL |
|Aspen      |CO   |USA    |ASE |
+-----------+-----+-------+----+
only showing top 20 rows



In [26]:
airportCodes.count

526

- All the unique IATA codes

In [18]:
airportCodes.select("IATA").
    distinct.
    count

524

- If they are not equal, analyze the data and identify IATA codes which are reapeated more than once

In [11]:
import org.apache.spark.sql.functions.{lit, count}

In [21]:
val duplicateIATACount = airportCodes.
    groupBy("IATA").
    agg(count(lit(1)).alias("iata_count")).
    filter($"iata_count" > 1).
    show(false)

+----+----------+
|IATA|iata_count|
+----+----------+
|Big |3         |
+----+----------+



duplicateIATACount: Unit = ()


In [22]:
airportCodes.
    filter($"IATA" === "Big").
    show(false)

+-----------+------+-------+----+
|City       |State |Country|IATA|
+-----------+------+-------+----+
|Hilo       |HI    |USA    |Big |
|Kailua-Kona|Hawaii|USA    |Big |
|Kamuela    |Hawaii|USA    |Big |
+-----------+------+-------+----+



In [23]:
airportCodes.
    filter($"State" =!= "Hawaii" and ($"IATA" === "Big")).
    show(false)

+----+-----+-------+----+
|City|State|Country|IATA|
+----+-----+-------+----+
|Hilo|HI   |USA    |Big |
+----+-----+-------+----+



In [24]:
val duplicateIATACount = airportCodes.
    groupBy("IATA").
    agg(count(lit(1)).alias("iata_count")).
    filter($"iata_count" > 1).
    show(false)

+----+----------+
|IATA|iata_count|
+----+----------+
|Big |3         |
+----+----------+



duplicateIATACount: Unit = ()


In [25]:
 airportCodes. 
    filter($"State" =!= "Hawaii" and ($"IATA" === "Big")).
    count

1

In [28]:
airportCodes.
    filter("!(State = 'Hawaii' AND IATA = 'Big')").
    count

524

 - Get the number of airports (IATA Codes) for each state in US. Sort the data in descending order by count.

In [32]:
val airportCodesPath = "/public/airlines_all/airport-codes"

airportCodesPath = /public/airlines_all/airport-codes


/public/airlines_all/airport-codes

In [35]:
val airportCodes = spark.
    read.
    option("sep", "\t").
    option("header", true).
    option("inferSchema", true).
    csv(airportCodesPath).
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country = 'USA'")    

airportCodes = [City: string, State: string ... 2 more fields]


lastException: Throwable = null


[City: string, State: string ... 2 more fields]

In [36]:
airportCodes.count

443

In [44]:
import org.apache.spark.sql.functions.{count, col, lit}

In [48]:
val airportCountByState = airportCodes.
    groupBy("Country", "State").
    agg(count(lit(1)).alias("IATACount")).
    orderBy($"IATACount".desc)

airportCountByState = [Country: string, State: string ... 1 more field]


[Country: string, State: string ... 1 more field]

In [49]:
airportCountByState.show(51, false)

+-------+-----+---------+
|Country|State|IATACount|
+-------+-----+---------+
|USA    |CA   |29       |
|USA    |TX   |26       |
|USA    |AK   |25       |
|USA    |NY   |18       |
|USA    |MI   |18       |
|USA    |FL   |18       |
|USA    |MT   |14       |
|USA    |PA   |13       |
|USA    |IL   |12       |
|USA    |CO   |12       |
|USA    |WY   |10       |
|USA    |NC   |10       |
|USA    |WI   |9        |
|USA    |NE   |9        |
|USA    |GA   |9        |
|USA    |NM   |9        |
|USA    |HI   |9        |
|USA    |WA   |9        |
|USA    |KS   |9        |
|USA    |ND   |8        |
|USA    |MO   |8        |
|USA    |AR   |8        |
|USA    |MA   |8        |
|USA    |MN   |8        |
|USA    |AZ   |8        |
|USA    |WV   |8        |
|USA    |IA   |8        |
|USA    |SD   |7        |
|USA    |ME   |7        |
|USA    |VA   |7        |
|USA    |LA   |7        |
|USA    |MS   |7        |
|USA    |OR   |7        |
|USA    |TN   |6        |
|USA    |AL   |6        |
|USA    |OH 

## Joining Data Sets

## Problem Statements

Let us understand how to join Data Frames by using some problem statements. We will use 2008 January airlines data along with Airport Codes

- Get the number of flights departed from each of the US airport.
- Get the number of flights departed from each of the state.
- Get the list of airports in the US from which flights are not departed.
- Check if there are any origins in airlines data which do not have record in airport-codes.
- Get the total number of the flights from the airports that do not contain entries in airport-codes.
- Get the total number of flights per airport that do not contain entries in airport-codes.

## Overview Of JOINS 

Let us get an overview of joining Data Frames.
 - Our data cannot be staored in one table. It will be stored in multiple tables and the tables might be related.
  - When it comes to transactional systems, we typically define tables based on Normalization
  - When it comes datewarehousing applications, we typically define tables using Dimensional Modeling.
  - Either of the approach data is scattered into multiple tables and relationships are defined.
  - Typically tables are related to one to one, one to many, many to many relationships.
 - When we have 2 Data Sets that are related based on common key we typically perform join.
 - There are different types of JOINS:
  - INNER JOIN
  - OUTER JOIN(LEFT or RIGHT)
  - FULL OUTER JOIN(a LEFT OUTER JOI b UNION a RIGHT OUTER JOIN b)

## Solution - Problem 1 

Get the number of flights departed from each of the US airport in the month of 2008 January.

 - We have to use airport codes to determine US airports.
 - We need to use airlines data to get departure details. 
 - To solve this problem we have to use INNER join.

In [5]:
val airlinesPath = "/public/airlines_all/airlines-part/flightmonth=200801"

airlinesPath = /public/airlines_all/airlines-part/flightmonth=200801


/public/airlines_all/airlines-part/flightmonth=200801

In [6]:
val airlines = spark.
    read.
    parquet(airlinesPath)

airlines = [Year: int, Month: int ... 29 more fields]


[Year: int, Month: int ... 29 more fields]

In [14]:
airlines.printSchema

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [16]:
airlines.select("Year", "Month", "DayofMonth", "Origin", "Dest", "CRSDepTime").show(false)

+----+-----+----------+------+----+----------+
|Year|Month|DayofMonth|Origin|Dest|CRSDepTime|
+----+-----+----------+------+----+----------+
|2008|1    |16        |BGR   |CVG |1735      |
|2008|1    |17        |SYR   |CVG |1701      |
|2008|1    |17        |SAV   |BOS |1225      |
|2008|1    |17        |CVG   |GRR |1530      |
|2008|1    |17        |STL   |CVG |1205      |
|2008|1    |18        |STL   |JFK |1150      |
|2008|1    |18        |MCI   |CVG |1009      |
|2008|1    |19        |TUL   |CVG |835       |
|2008|1    |20        |JFK   |PHL |1935      |
|2008|1    |20        |RDU   |CVG |830       |
|2008|1    |21        |CVG   |DTW |1640      |
|2008|1    |21        |MSY   |LGA |1204      |
|2008|1    |21        |JFK   |PHL |1935      |
|2008|1    |21        |DCA   |JFK |1830      |
|2008|1    |21        |HSV   |DCA |700       |
|2008|1    |22        |ORD   |CVG |1910      |
|2008|1    |22        |CVG   |JFK |1320      |
|2008|1    |23        |LGA   |SAV |908       |
|2008|1    |2

In [17]:
airlines.count

605659

In [7]:
val airportCodesPath = "/public/airlines_all/airport-codes"

airportCodesPath = /public/airlines_all/airport-codes


/public/airlines_all/airport-codes

In [58]:
"hdfs dfs -ls /public/airlines_all/airport-codes" !

Found 1 items
-rw-r--r--   2 hdfs hdfs      11411 2019-11-26 18:32 /public/airlines_all/airport-codes/airport-codes-na.txt


0

In [8]:
def getValidAirportCodes(airportCodesPath: String) = {
    val airportCodes = spark.
    read.
    option("sep", "\t").
    option("header", true).
    option("inferSchema", true).
    csv(airportCodesPath).
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country = 'USA'")
    airportCodes   
}

getValidAirportCodes: (airportCodesPath: String)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [13]:
val airportCodes = getValidAirportCodes(airportCodesPath)

airportCodes = [City: string, State: string ... 2 more fields]


[City: string, State: string ... 2 more fields]

In [14]:
airportCodes.count

443

In [15]:
import org.apache.spark.sql.functions.{col, lit, count}

In [22]:
airlines.
    join(airportCodes, airportCodes("IATA") === airlines("Origin") ).
    select(col("Year"), col("Month"), col("DayOfMonth"), airportCodes("*"), col("CRSDepTime")).
    show(false)

+----+-----+----------+-------------+-----+-------+----+----------+
|Year|Month|DayOfMonth|City         |State|Country|IATA|CRSDepTime|
+----+-----+----------+-------------+-----+-------+----+----------+
|2008|1    |16        |Bangor       |ME   |USA    |BGR |1735      |
|2008|1    |17        |Syracuse     |NY   |USA    |SYR |1701      |
|2008|1    |17        |Savannah     |GA   |USA    |SAV |1225      |
|2008|1    |17        |Cincinnati   |OH   |USA    |CVG |1530      |
|2008|1    |17        |St. Louis    |MO   |USA    |STL |1205      |
|2008|1    |18        |St. Louis    |MO   |USA    |STL |1150      |
|2008|1    |18        |Kansas City  |MO   |USA    |MCI |1009      |
|2008|1    |19        |Tulsa        |OK   |USA    |TUL |835       |
|2008|1    |20        |New York     |NY   |USA    |JFK |1935      |
|2008|1    |20        |Raleigh      |NC   |USA    |RDU |830       |
|2008|1    |21        |Cincinnati   |OH   |USA    |CVG |1640      |
|2008|1    |21        |New Orleans  |LA   |USA  

In [25]:
airlines.
    join(airportCodes, airportCodes("IATA") === airlines("Origin") ).
    groupBy("Origin").
    agg(count(lit(1)).alias("FlightCount")).
    orderBy(col("FlightCount").desc).
    show(false)

+------+-----------+
|Origin|FlightCount|
+------+-----------+
|ATL   |33897      |
|ORD   |29936      |
|DFW   |23861      |
|DEN   |19477      |
|LAX   |18945      |
|PHX   |17695      |
|IAH   |15531      |
|LAS   |15292      |
|DTW   |14357      |
|EWR   |12467      |
|SLC   |12401      |
|MSP   |11800      |
|SFO   |11573      |
|MCO   |11070      |
|CLT   |10752      |
|LGA   |10300      |
|JFK   |10023      |
|BOS   |9717       |
|BWI   |8883       |
|CVG   |8659       |
+------+-----------+
only showing top 20 rows



lastException: Throwable = null


## Solution - Problem 2

Get numbers of flights departed from each of the US state in the month of 2008 January

- We need to use airport codes to determine state of each the US airport.
- We need to use airlines data to get departure details.
- To solve this problem we have to perform inner join.

In [10]:
val airlinesPath = "/public/airlines_all/airlines-part/flightmonth=200801"

airlinesPath = /public/airlines_all/airlines-part/flightmonth=200801


/public/airlines_all/airlines-part/flightmonth=200801

In [11]:
val airlines = spark.
    read.
    parquet(airlinesPath)

airlines = [Year: int, Month: int ... 29 more fields]


[Year: int, Month: int ... 29 more fields]

In [12]:
val airportCodesPath = "/public/airlines_all/airport-codes"

airportCodesPath = /public/airlines_all/airport-codes


/public/airlines_all/airport-codes

In [13]:
def getValidAirportCodes(airportCodesPath: String) = {
    val airportCodes = spark.
    read.
    option("sep", "\t").
    option("header", true).
    option("inferSchema", true).
    csv(airportCodesPath).
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country = 'USA'")
    airportCodes   
}

getValidAirportCodes: (airportCodesPath: String)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [18]:
val airportCodes = getValidAirportCodes(airportCodesPath)

airportCodes = [City: string, State: string ... 2 more fields]


[City: string, State: string ... 2 more fields]

In [15]:
import org.apache.spark.sql.functions.{col, lit, count}

In [26]:
airlines.
    join(airportCodes, col("IATA") === col("Origin"), "inner").
    groupBy("State").
    agg(count(lit(1)).alias("FlightCount")).
    orderBy(col("FlightCount").desc).
    show

+-----+-----------+
|State|FlightCount|
+-----+-----------+
|   CA|      72853|
|   TX|      63930|
|   FL|      41042|
|   IL|      39812|
|   GA|      35527|
|   NY|      28414|
|   CO|      23288|
|   AZ|      20768|
|   OH|      19209|
|   NC|      17942|
|   MI|      17824|
|   NV|      17763|
| null|      14090|
|   TN|      13549|
|   PA|      13491|
|   UT|      12709|
|   NJ|      12498|
|   MN|      12357|
|   MO|      11808|
|   WA|      10210|
+-----+-----------+
only showing top 20 rows



In [21]:
airportCodes.show(false)

+-------------+-----+-------+----+
|City         |State|Country|IATA|
+-------------+-----+-------+----+
|Aberdeen     |SD   |USA    |ABR |
|Abilene      |TX   |USA    |ABI |
|Akron        |OH   |USA    |CAK |
|Alamosa      |CO   |USA    |ALS |
|Albany       |GA   |USA    |ABY |
|Albany       |NY   |USA    |ALB |
|Albuquerque  |NM   |USA    |ABQ |
|Alexandria   |LA   |USA    |AEX |
|Allentown    |PA   |USA    |ABE |
|Alliance     |NE   |USA    |AIA |
|Alpena       |MI   |USA    |APN |
|Altoona      |PA   |USA    |AOO |
|Amarillo     |TX   |USA    |AMA |
|Anchorage    |AK   |USA    |ANC |
|Appleton     |WI   |USA    |ATW |
|Asheville    |NC   |USA    |AVL |
|Aspen        |CO   |USA    |ASE |
|Athens       |GA   |USA    |AHN |
|Atlanta      |GA   |USA    |ATL |
|Atlantic City|NJ   |USA    |ACY |
+-------------+-----+-------+----+
only showing top 20 rows



In [22]:
airportCodes.filter("State IS NULL").show

+-------------+-----+-------+----+
|         City|State|Country|IATA|
+-------------+-----+-------+----+
|Washington DC| null|    USA| IAD|
|Washington DC| null|    USA| DCA|
|Washington DC| null|    USA| WAS|
+-------------+-----+-------+----+



In [25]:
airportCodes.filter(col("State") isNull).show

+-------------+-----+-------+----+
|         City|State|Country|IATA|
+-------------+-----+-------+----+
|Washington DC| null|    USA| IAD|
|Washington DC| null|    USA| DCA|
|Washington DC| null|    USA| WAS|
+-------------+-----+-------+----+



## Solution - Problem 3

Get the list of airports in the US from which flights are not departed in the month of 2008 January

- This is an example of outer join.
- We need to get those airports which are in airport codes but not in 2008 January airlines data set.
- Based on the side of airport codes data set, we can say left or right. We will be using airport codes as the driving data set and hence we will use left outer join

In [13]:
import org.apache.spark.sql.functions.{col, lit, count}

In [5]:
val airlinesPath = "/public/airlines_all/airlines-part/flightmonth=200801"

airlinesPath = /public/airlines_all/airlines-part/flightmonth=200801


/public/airlines_all/airlines-part/flightmonth=200801

In [8]:
val airlines = spark.
    read.
    parquet(airlinesPath)

airlines = [Year: int, Month: int ... 29 more fields]


[Year: int, Month: int ... 29 more fields]

In [9]:
val airportCodesPath = "/public/airlines_all/airport-codes"

airportCodesPath = /public/airlines_all/airport-codes


/public/airlines_all/airport-codes

In [10]:
def getValidAirportCodes(airportCodesPath: String) = {
    val airportCodes = spark.
    read.
    option("sep", "\t").
    option("header", true).
    option("inferSchema", true).
    csv(airportCodesPath).
    filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country = 'USA'")
    airportCodes   
}

getValidAirportCodes: (airportCodesPath: String)org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]


In [11]:
val airportCodes = getValidAirportCodes(airportCodesPath)

airportCodes = [City: string, State: string ... 2 more fields]


[City: string, State: string ... 2 more fields]

In [15]:
airportCodes.count

443

In [17]:
airportCodes.
    join(airlines, col("IATA") === col("Origin"), "left").
    filter("Origin IS NULL").
    select(airportCodes("*"), col("Origin")).
    show(false)

+--------------+-----+-------+----+------+
|City          |State|Country|IATA|Origin|
+--------------+-----+-------+----+------+
|Aberdeen      |SD   |USA    |ABR |null  |
|Alamosa       |CO   |USA    |ALS |null  |
|Alliance      |NE   |USA    |AIA |null  |
|Alpena        |MI   |USA    |APN |null  |
|Altoona       |PA   |USA    |AOO |null  |
|Athens        |GA   |USA    |AHN |null  |
|Augusta       |ME   |USA    |AUG |null  |
|Bar Harbor    |ME   |USA    |BHB |null  |
|Beckley       |WV   |USA    |BKW |null  |
|Bedford       |MA   |USA    |BED |null  |
|Bemidji       |MN   |USA    |BJI |null  |
|Bettles       |AK   |USA    |BTT |null  |
|Bloomington   |IN   |USA    |BMG |null  |
|Bluefield     |WV   |USA    |BLF |null  |
|Brookings     |SD   |USA    |BKX |null  |
|Burlington    |IA   |USA    |BRL |null  |
|Burlington    |MA   |USA    |BBF |null  |
|Cape Girardeau|MO   |USA    |CGI |null  |
|Carlsbad      |NM   |USA    |CNM |null  |
|Cheyenne      |WY   |USA    |CYS |null  |
+----------